In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

iris = load_iris()
X, y = iris.data, iris.target

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

dense_model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(4,)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

dense_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history_dense = dense_model.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test))

loss, acc = dense_model.evaluate(X_test, y_test)
print(f"Dokładność na zbiorze testowym (Dense IRIS): {acc:.2f}")


(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train[..., tf.newaxis] / 255.0
x_test = x_test[..., tf.newaxis] / 255.0

cnn_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28,28,1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history_cnn = cnn_model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

loss, acc = cnn_model.evaluate(x_test, y_test)
print(f"Dokładność na zbiorze testowym (CNN MNIST): {acc:.2f}")

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=10000)

x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen=500)
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test, maxlen=500)

gru_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=10000, output_dim=32),
    tf.keras.layers.GRU(64),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

gru_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history_gru = gru_model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

loss, acc = gru_model.evaluate(x_test, y_test)
print(f"Dokładność na zbiorze testowym (GRU IMDB): {acc:.2f}")

X = np.random.rand(1000, 10, 512)
Y = np.random.rand(1000, 512)

def build_transformer_encoder(num_layers=2, num_heads=2, key_dim=64, input_shape=(10, 512)):
    inputs = tf.keras.layers.Input(shape=input_shape)
    x = inputs
    for _ in range(num_layers):
        attn_output = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)(x, x)
        attn_output = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x + attn_output)
        ffn_output = tf.keras.layers.Dense(512, activation='relu')(attn_output)
        x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attn_output + ffn_output)
    outputs = tf.keras.layers.GlobalAveragePooling1D()(x)
    outputs = tf.keras.layers.Dense(512, activation='linear')(outputs)
    return tf.keras.Model(inputs=inputs, outputs=outputs)

transformer_model = build_transformer_encoder()

transformer_model.compile(optimizer='adam', loss='mse')

history_transformer = transformer_model.fit(X, Y, epochs=10, batch_size=32)

loss = transformer_model.evaluate(X, Y)
print(f"Błąd średniokwadratowy na danych testowych (Transformer): {loss:.4f}")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 178ms/step - accuracy: 0.4087 - loss: 1.0933 - val_accuracy: 0.6333 - val_loss: 1.0087
Epoch 2/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 93ms/step - accuracy: 0.4658 - loss: 1.0431 - val_accuracy: 0.7000 - val_loss: 0.9594
Epoch 3/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.5244 - loss: 0.9831 - val_accuracy: 0.7000 - val_loss: 0.9115
Epoch 4/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.6515 - loss: 0.9336 - val_accuracy: 0.7000 - val_loss: 0.8671
Epoch 5/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.5858 - loss: 0.9107 - val_accuracy: 0.7000 - val_loss: 0.8228
Epoch 6/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.6796 - loss: 0.8474 - val_accuracy: 0.7000 - val_loss: 0.7779
Epoch 7/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.6715 - loss: 0.8381 - val_accuracy: 0.7000 - val_loss: 0.7333
Epoch 8/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.6862 - loss: 0.7936 - val_accuracy: 0.7667 - val_loss: 0.6873

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 75s 39ms/step - accuracy: 0.9342 - loss: 0.2210 - val_accuracy: 0.9751 - val_loss: 0.0820
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 74s 39ms/step - accuracy: 0.9871 - loss: 0.0425 - val_accuracy: 0.9880 - val_loss: 0.0382
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 79s 38ms/step - accuracy: 0.9911 - loss: 0.0284 - val_accuracy: 0.9848 - val_loss: 0.0478
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 70s 37ms/step - accuracy: 0.9923 - loss: 0.0251 - val_accuracy: 0.9879 - val_loss: 0.0443
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 72s 38ms/step - accuracy: 0.9945 - loss: 0.0168 - val_accuracy: 0.9877 - val_loss: 0.0485
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9845 - loss: 0.0609
Dokładność na zbiorze testowym (CNN MNIST): 0.99
17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 327s 414ms/step - accuracy: 0.6936 - loss: 0.5490 - val_accuracy: 0.7388 - val_loss: 0.5196
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 